In [2]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import tagui as t
import urllib
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [3]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [4]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows

In [5]:
site = getSiteByName("New York Times")
site_url = site[0][2]
site_id = site[0][0]

In [6]:
t.init(visual_automation = True, chrome_browser = True)
t.url(site_url)

[RPA][ERROR] - cannot find //div[@class="container footer-main"]


True

In [7]:
number_to = t.count('(//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")])')

In [18]:
def remove_str(summm_l, SS):
    summm = ''.join(summm_l)
    summm = summm.encode('utf-8').decode('utf-8', 'ignore')
    if SS in summm:
        summm = summm.replace(SS, '')
    
    return summm

for i in range(0, df.shape[0]):
    df['Summary'][i] = remove_str(df['Summary'][i], "u'")

c:\users\superhell\.conda\envs\ca1\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [19]:
df

,Sno,Title,URL,Summary,Img_URL
0,0,Trump Moves to Replace Watchdog Who Identified...,https://www.nytimes.com/2020/05/01/us/politics...,"This is a difficult question, because a lot de...",https://static01.nyt.com/images/2020/05/01/mul...
1,1,What Is ‘Covid Toe’? Maybe a Strange Sign of C...,https://www.nytimes.com/2020/05/01/health/coro...,"Over the past few weeks, she has seen dozens.\...",https://static01.nyt.com/images/2020/04/30/sci...
2,2,"Virus Deaths in New York Increase Slightly, Cu...",https://www.nytimes.com/2020/05/02/nyregion/co...,"“We are trying to understand why that is,” he ...",https://static01.nyt.com/images/2020/05/02/nyr...
3,3,Investigate Tara Reade’s Allegations,https://www.nytimes.com/2020/05/01/opinion/bid...,"Biden said that if such a document existed, th...",https://static01.nyt.com/images/2020/05/02/opi...
4,4,Coronavirus Live Updates: States Rely on Resid...,https://www.nytimes.com/2020/05/02/us/coronavi...,"In places still under lockdown, the weekend is...",https://static01.nyt.com/images/2020/05/02/us/...
5,5,The New Great Depression Is Coming. Will There...,https://www.nytimes.com/2020/05/02/opinion/sun...,The New Great Depression Is Coming.Will There ...,https://static01.nyt.com/images/2020/05/02/opi...
6,6,What the Proponents of ‘Natural’ Herd Immunity...,https://www.nytimes.com/2020/05/01/opinion/cor...,Some countries are attempting strategies inten...,https://static01.nyt.com/images/2020/05/03/opi...
7,7,Olympic Gymnast Recalls Emotional Abuse ‘So Tw...,https://www.nytimes.com/2020/05/01/sports/magg...,"Coach Maggie Haney, left, watching Laurie Hern...",https://static01.nyt.com/images/2020/05/02/spo...
8,8,Migrants Are Rounded Up in Malaysia and Face S...,https://www.nytimes.com/2020/05/02/world/coron...,The companies insist they are not profiteering...,https://static01.nyt.com/images/2020/05/02/wor...
9,9,My Retirement Plan Is You,https://www.nytimes.com/2020/05/02/business/Pa...,Browsing a Salvation Army store.Ms.Danigelis w...,https://static01.nyt.com/images/2020/04/30/bus...


In [20]:
t.close()

True

In [21]:
def insertNews(conn,news):    
    sql = ''' INSERT INTO news(site_id,create_date,title,url,summary,img_url,latest)
              VALUES(?,?,?,?,?,?,1) '''
    cur = conn.cursor()
    cur.execute(sql, news)
    return cur.lastrowid

def markNews(conn, site_id):
    cur = conn.cursor()
    cur.execute("UPDATE news set latest=0 where latest=1 and site_id=? ",(site_id,))

In [24]:
now = datetime.now()
# dd/mm/YY
dt_string = now.strftime("%d/%m/%Y")
markNews(conn, site_id)
for index, row in df.iterrows():
    title = row['Title'].decode('utf-8')
    url = row['URL']
    summary = row['Summary']
    img_url = row['Img_URL']
    news = (site_id,dt_string,title,url,summary,img_url)
    news_id = insertNews(conn,news)
    print("News %d has been created" % news_id)
    
conn.commit()

News 162 has been created
News 163 has been created
News 164 has been created
News 165 has been created
News 166 has been created
News 167 has been created
News 168 has been created
News 169 has been created
News 170 has been created
News 171 has been created
News 172 has been created


In [11]:
conn.close()